   ## If your model is scored with some metric, you get best results by optimizing exactly this metric

### Exploritary metric analysis

#### 1) Regression
     * MSE, RMSE, R-squared
       MSE, Best constant for base model - target mean value

<img src="files/Images/MSE.png" width="400" height="100">

       RMSE - skale for the error is the same as target values. We can optimize MSE instead of RMSE. 
       
<img src="files/Images/RMSE.png" width="400" height="100">

        R-squered used to measure relults in range. To optimize R-squared we can optimize MSE.
        
<img src="files/Images/rsquered.png" width="400" height="100">
        
     * MAE
     Not that sensetive to outliets as MSE. Used often in Finance. Best constant for base model - Median. More robust than MSE.
     
<img src="files/Images/MAE.png" width="400" height="100">

    Non-deferrable at zero


<img src="files/Images/MAE_grad.png" width="400" height="100">
       
       
   #### total on mse/mae
    Do you Have outliers in the data?
        Use MAE
    Are you sure they are outliers?
        Use MAE
    Or they are just unexpected values we should still care about?
        Use MSE
        
     * (R)MSPE, MAPE
     * (R)MSLE

#### 2) Classification
      * Accuracy, LogLoss, AUC
      * Cohen's(Quadratic weighted) Kappa